In [1]:
# train_los_models.py

# Step 0: Packages and setup
import os
import numpy as np
import pandas as pd
from math import sqrt
from pathlib import Path
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
)
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
import joblib

# -----------------------------
# Config
# -----------------------------
RANDOM_STATE = 100
TRAIN_PCT = 0.70
file_path = Path("..") / "Data" / "LengthOfStay.csv"  # adjust if needed

TARGET = "lengthofstay"
ID_COL = "eid"
DATE_COLS = ["vdate", "discharged"]
DROP_FROM_FEATURES = ["eid", "vdate", "discharged", "facid"]  # match original exclusions

# Columns standardized in the original notebook (Step 2)
CONTINUOUS_TO_STANDARDIZE = [
    "hematocrit",
    "neutrophils",
    "sodium",
    "glucose",
    "bloodureanitro",
    "creatinine",
    "bmi",
    "pulse",
    "respiration",
]

# Indicator columns used to compute "number_of_issues" (Step 2)
ISSUE_INDICATORS = [
    "hemo",
    "dialysisrenalendstage",
    "asthma",
    "irondef",
    "pneum",
    "substancedependence",
    "psychologicaldisordermajor",
    "depress",
    "psychother",
    "fibrosisandother",
    "malnutrition",
]

# Reasonable guesses for a "procedure" column
PROCEDURE_COL_CANDIDATES = [
    "procedure",
    "primaryprocedure",
    "surgery",
    "aprdrgdescription",
]

# -----------------------------
# Utility: evaluation
# -----------------------------
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print(f"[{model_name}] MAE={mae:.4f} | RMSE={rmse:.4f} | R^2={r2:.4f}")
    return {"model": model_name, "MAE": mae, "RMSE": rmse, "R2": r2}


# -----------------------------
# Step 1: Load, type cast, clean NA
# -----------------------------
df = pd.read_csv(file_path)

# Parse dates
for c in DATE_COLS:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

# Make sure indicator columns exist and are numeric
present_issue_cols = [c for c in ISSUE_INDICATORS if c in df.columns]
for c in present_issue_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Identify columns to clean (no NA cleaning on eid/lengthofstay/dates)
protected = {ID_COL, TARGET, *DATE_COLS}
cols_to_consider = [c for c in df.columns if c not in protected]

num_cols_all = df[cols_to_consider].select_dtypes(include=["number"]).columns.tolist()
cat_cols_all = df[cols_to_consider].select_dtypes(include=["object", "category"]).columns.tolist()

# Fill numeric with mean
if num_cols_all:
    means = df[num_cols_all].mean(numeric_only=True)
    df[num_cols_all] = df[num_cols_all].fillna(means)

# Fill categorical with mode
for c in cat_cols_all:
    if df[c].isna().any():
        mode_val = df[c].mode(dropna=True)
        if not mode_val.empty:
            df[c] = df[c].fillna(mode_val.iloc[0])
        else:
            df[c] = df[c].fillna("UNKNOWN")

print("Step 1 complete: data cleaned (numeric->mean, categorical->mode).")

# -----------------------------
# Step 2: Feature engineering
# -----------------------------
present_cont = [c for c in CONTINUOUS_TO_STANDARDIZE if c in df.columns]
for c in present_cont:
    std = df[c].std(ddof=0)
    mean = df[c].mean()
    df[c] = (df[c] - mean) / (std if std and std != 0 else 1.0)

# number_of_issues = sum of indicator columns
if present_issue_cols:
    df["number_of_issues"] = (
        df[present_issue_cols]
        .apply(pd.to_numeric, errors="coerce")
        .fillna(0)
        .sum(axis=1)
        .astype(int)
    )
    df["number_of_issues"] = df["number_of_issues"].astype(str)
else:
    df["number_of_issues"] = "0"

# Ensure target is numeric
df[TARGET] = pd.to_numeric(df[TARGET], errors="coerce")

print("Step 2 complete: standardized selected labs and created number_of_issues.")

# -----------------------------
# Procedure stats (for avg LoS, distribution, etc.)
# -----------------------------
PROC_COL = next((c for c in PROCEDURE_COL_CANDIDATES if c in df.columns), None)
if PROC_COL:
    proc_stats = df.groupby(PROC_COL)[TARGET].agg(["mean", "std", "count"])
    print(f"Using procedure column: {PROC_COL}")
else:
    proc_stats = None
    print("No procedure column found among candidates; procedure-based stats disabled.")

# -----------------------------
# Prepare features/target and split
# -----------------------------
feature_cols = [c for c in df.columns if c not in set(DROP_FROM_FEATURES + [TARGET])]
X = df[feature_cols].copy()
y = df[TARGET].copy()

cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(exclude=["object", "category"]).columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=TRAIN_PCT, random_state=RANDOM_STATE
)

print(f"Split complete: train={len(X_train)} rows, test={len(X_test)} rows.")

# ColumnTransformer: One-hot encode categoricals, pass numeric through
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
        ("num", "passthrough", num_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

# -----------------------------
# Step 3: Train & evaluate models
# -----------------------------
results = []
oob_min_samples_split = int(max(2, round(sqrt(len(X_train)))))

models = {
    "RF": RandomForestRegressor(
        n_estimators=40,
        random_state=5,
        oob_score=True,
        bootstrap=True,
        n_jobs=-1,
        min_samples_split=oob_min_samples_split,
    ),
    "GBT": GradientBoostingRegressor(
        n_estimators=40,
        learning_rate=0.3,
        random_state=9,
    ),
    "FastTrees": HistGradientBoostingRegressor(
        max_iter=100,
        learning_rate=0.1,
        random_state=RANDOM_STATE,
    ),
    "NN": MLPRegressor(
        hidden_layer_sizes=(64, 64),
        activation="relu",
        solver="adam",
        max_iter=500,
        random_state=17,
    ),
    # Simple interpretable model for per-feature contributions
    "LIN": LinearRegression(),
}

fitted_pipelines = {}
for name, model in models.items():
    pipe = Pipeline(steps=[("prep", preprocess), ("model", model)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    fitted_pipelines[name] = pipe

    res = evaluate_model(y_test, y_pred, name)
    if name == "RF" and hasattr(pipe.named_steps["model"], "oob_score_"):
        print(f"[RF] OOB R^2={pipe.named_steps['model'].oob_score_:.4f}")
    results.append(res)

# Collate results
metrics_df = pd.DataFrame(results).sort_values("RMSE")
print("\nSummary metrics (sorted by RMSE):")
print(metrics_df)

# Optionally persist models
out_dir = Path("./models_sklearn_los")
out_dir.mkdir(parents=True, exist_ok=True)
for name, pipe in fitted_pipelines.items():
    joblib.dump(pipe, out_dir / f"{name}_model.joblib")

# -----------------------------
# Build predictions tables
# -----------------------------
def build_predictions_table(model_key="GBT"):
    assert model_key in fitted_pipelines, f"Unknown model_key {model_key}"
    pipe = fitted_pipelines[model_key]

    pred = pipe.predict(X_test)
    pred_round = np.rint(pred).astype(int)

    include_cols = [
        "eid",
        "vdate",
        "rcount",
        "gender",
        "dialysisrenalendstage",
        "asthma",
        "irondef",
        "pneum",
        "substancedependence",
        "psychologicaldisordermajor",
        "depress",
        "psychother",
        "fibrosisandother",
        "malnutrition",
        "hemo",
        "hematocrit",
        "neutrophils",
        "sodium",
        "glucose",
        "bloodureanitro",
        "creatinine",
        "bmi",
        "pulse",
        "respiration",
        "number_of_issues",
        "secondarydiagnosisnonicd9",
        "discharged",
        "facid",
        "lengthofstay",
    ]
    if PROC_COL:
        include_cols.append(PROC_COL)

    present_cols = [c for c in include_cols if c in df.columns]

    base = df.loc[X_test.index, present_cols].copy()

    if "vdate" in base.columns and not np.issubdtype(base["vdate"].dtype, np.datetime64):
        base["vdate"] = pd.to_datetime(base["vdate"], errors="coerce")

    base["lengthofstay_Pred"] = pred
    base["lengthofstay_Pred_Rounded"] = pred_round
    if "vdate" in base.columns:
        base["discharged_Pred"] = base["vdate"] + pd.to_timedelta(
            base["lengthofstay_Pred_Rounded"], unit="D"
        )
    else:
        base["discharged_Pred"] = pd.NaT

    # Add procedure-level context if available
    if PROC_COL and proc_stats is not None and PROC_COL in base.columns:
        base["procedure_AvgLoS"] = base[PROC_COL].map(proc_stats["mean"])
        base["procedure_LoS_Std"] = base[PROC_COL].map(proc_stats["std"])

        # Simple z-score based outlier flag
        with np.errstate(divide="ignore", invalid="ignore"):
            z = (base["lengthofstay_Pred"] - base["procedure_AvgLoS"]) / base[
                "procedure_LoS_Std"
            ]
        base["procedure_LoS_Z"] = z
        base["possible_Outlier"] = z.abs() > 2

    # Ordering
    order = [
        "eid",
        "vdate",
        "lengthofstay",
        "lengthofstay_Pred",
        "lengthofstay_Pred_Rounded",
        "discharged_Pred",
    ]
    if PROC_COL and PROC_COL in base.columns:
        order.append(PROC_COL)
        order += [
            "procedure_AvgLoS",
            "procedure_LoS_Std",
            "procedure_LoS_Z",
            "possible_Outlier",
        ]
    order += [c for c in present_cols if c not in order]
    cols_final = [c for c in order if c in base.columns]
    return base[cols_final]


los_predictions_gbt = build_predictions_table("GBT")
los_predictions_rf = build_predictions_table("RF")
los_predictions_fast = build_predictions_table("FastTrees")
los_predictions_nn = build_predictions_table("NN")
los_predictions_lin = build_predictions_table("LIN")

# Save predictions
out_dir_preds = Path("./predictions_sklearn_los")
out_dir_preds.mkdir(parents=True, exist_ok=True)
los_predictions_gbt.to_csv(out_dir_preds / "LoS_Predictions_GBT.csv", index=False)
los_predictions_rf.to_csv(out_dir_preds / "LoS_Predictions_RF.csv", index=False)
los_predictions_fast.to_csv(out_dir_preds / "LoS_Predictions_FastTrees.csv", index=False)
los_predictions_nn.to_csv(out_dir_preds / "LoS_Predictions_NN.csv", index=False)
los_predictions_lin.to_csv(out_dir_preds / "LoS_Predictions_LIN.csv", index=False)

print("\nPredictions tables written to ./predictions_sklearn_los/")


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Step 1 complete: data cleaned (numeric->mean, categorical->mode).
Step 2 complete: standardized selected labs and created number_of_issues.
No procedure column found among candidates; procedure-based stats disabled.
Split complete: train=70000 rows, test=30000 rows.
[RF] MAE=0.4940 | RMSE=0.7880 | R^2=0.8891
[RF] OOB R^2=0.8905
[GBT] MAE=0.3675 | RMSE=0.4854 | R^2=0.9579
[FastTrees] MAE=0.3110 | RMSE=0.4310 | R^2=0.9668
[NN] MAE=0.3253 | RMSE=0.4753 | R^2=0.9597
[LIN] MAE=0.8449 | RMSE=1.1047 | 